# Item 18: Know How to Construct Key-Dependent Default Values with `__missing__`

In [1]:
# There are situations when neither setdefault nor defaultdict is the right fit
# Using a normal dict instance and checking for the presence of keys using the get method and an assignment
# expression
pictures = {}
path = 'profile_1234.png'

if (handle := pictures.get(path)) is None:
    try:
        handle = open(path, 'a+b')
    except OSError:
        print(f'Failed to open path {path}')
        raise
    else:
        pictures[path] = handle

handle.seek(0)
image_data = handle.read()

In the code above, when the file handle already exists in the dictionary, this code makes only a single dictionary access. In the case that the file handle doesn't exist, the dictionary is accessed once by `get`, and then it is assigned in the else clause of the `try/except` block. The call to the `read` method stands clearly separate from the code that calls `open` and handles exceptions.

In [2]:
# Its possible to use the in expression or KeyError approaches but these options require more dictionary access
# and levels of nesting. You might think that the setdefault method works too, since the other two approaches
# work
try:
    handle = pictures.setdefault(path, open(path, 'a+b'))
except OSError:
    print(f'Failed to open {path}')
    raise
else:
    handle.seek(0)
    image_data = handle.read()

The code above has many problems. The `open` built-in function to create the file handle is always called, even when the path is already present in the dictionary. This results in an additional file handle that may conflict with existing open handles in the same program. Exceptions may be raised by the `open` call and need to be handled, but it may not be possible to differetiate them from exceptions that may be raised by the `setdefault` call on the same line (which is possible for other dictionary-like implementations)

In [3]:
# If we're trying to manage internal state, another assumption you might make is that a defaultdict could be used
# for keeping track of these profile pictures. Here we try to implement the same logic as before but now using
# a helper function and the defaultdict class
from collections import defaultdict

def open_picture(profile_path):
    try:
        return open(profile_path, 'a+b')
    except OSError:
        print(f'Failed to open path {profile_path}')
        raise

pictures = defaultdict(open_picture)
handle = pictures[path]
handle.seek(0)
image_data = handle.read()

TypeError: open_picture() missing 1 required positional argument: 'profile_path'

The problem with the code above is that `defaultdict` expects that the function passed to its contructor doesn't require any arguments. This means that the helper function that `defaultdict` calls doesn't know which specific key is being accessed, which eliminates our ability to call `open`. In this particular situationm both `setdefault` and `defaultdict` fall short of what we need.

Fortunately, Python offers us a way out of this. We can subclass the `dict` typ and implement the `__missing__` special method to add custom logic for handling missing keys.

In [4]:
# Here we do the above by defining a new class that takes advantage of the same open_picture helper method
# defined previously
class Pictures(dict):
    def __missing__(self, key):
        value = open_picture(key)
        self[key] = value
        return value

pictures = Pictures()
handle = pictures[path]
handle.seek(0)
image_data = handle.read()

In the code above, when the `pictures[path]` dictionary access finds that the `path` key isn't present in the dictionary, the `__missing__` method is called. This method must create the new default value for the key, insert it into the dictionary, and return it to the caller. Subsequent accesses of the same path will not call `__missing__` since the corresponding item is already present. 